# Imports

In [159]:
import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import (RandomTreesEmbedding, IsolationForest, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder    
import json
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.model_selection import train_test_split
import random
from sklearn import preprocessing
from random import randint
import numpy as np
import os
from sklearn import decomposition

# Definisions

In [160]:
dataPrefix = '/home/data/aaia17/'
dataFeaturesOutput = '/home/data/aaia17/features/'
def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
def printify(t):
    for x in t:
        print(x)
def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)    
def readJsonData(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        with open(fData, 'r') as f:
            frame.append(json.load(f))
        f.close()
    return frame  
def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature
def dataScaling(data, scaler):
    data = data/(data+1)
    return scaler.fit_transform(data)
def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)
def readFileByLine(fileName):
    arr = []
    with open(fileName, 'r') as f:        
        for line in f:
            arr.append(line.rstrip())
        f.close()
    return arr
def converseVec2Matrix(data):
    arr = np.empty((len(data), 1), float)
    for i in range(0, len(data)):
        arr[i] = data[i]
    return arr   
def read_from_pickle(path):
    with open(path, 'rb') as file:
        try:
            return pickle.load(file)
        except EOFError:
            pass     
def meanVector(filenames):
    sumPredicted = np.zeros(750000)
    for filename in filenames:
        with open(averageDir+filename) as file:
            w = np.array(map(lambda x: float(x), file.readlines()))
            sumPredicted += w     
        file.close()
    return sumPredicted/len(filenames)
def replaceNan(data, item):
    return [x if x is not None else item for x in data]

def getDenseFeatures(dataFrame, names, dictionary):
    X_dense = np.empty((len(dataFrame), 0), float)
    for name in names:
        values = dataFrame[name].values
        X_dense = np.column_stack([X_dense, replaceNan([dictionary[name].get(value) for value in values], 0.5)])
    return X_dense
def addNewFeature(data, vectors):
    return np.column_stack([data, vectors])
def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature
def computeFractionsForAll(dataFrame, names):
    changeDict = dict()
    for name in names:
        changeDict[name] = _computeFractionForAttribute(dataFrame, name)
    return changeDict
def dropColumnsFromDataFrame(df, columns):
    for column in columns:
        df = df.drop(column, 1)
    return df
def readJsonData(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        with open(fData, 'r') as f:
            frame.append(json.load(f))
        f.close()
    return frame  
def readJsonDatabyLine(prefix, fNames):
    frame = []
    for fileName in fNames:
        fData = prefix+fileName
        frame.extend(readFileByLine(fData))
    return frame
def readJsonFromString(data):
    result = []
    nulls = []
    for i in range(0,len(data)):
        result.append(json.loads(data[i]))
                    #nulls.append(i)
    return result
def weighting(x):
    return np.exp(-(np.power(xaxis,4))/2.0)       
def frange(x0, x1, dx, fract):
    return np.array(map(lambda x: x / float(fract), range(int(fract*x0), int(fract*x1), int(fract*(dx)))))
def meanVectorW(pref, ids, weights):
    sumPredicted = np.zeros(750000)
    i = 0
    for id in ids:
        with open(dataPrefix+"%s%d.txt"%(pref, id)) as file:
            w = weights[i]*np.array(map(lambda x: float(x), file.readlines()))
            sumPredicted += w     
        file.close()
        i+=1
    return sumPredicted/np.sum(weights)

# Feature Engineering

In [ ]:
def getHandHpHistogram(state):
    handHp = np.zeros(3)
    # maxval is 8, we divide by 3
    for hand in state['player']['hand']:
        try:
            handHp[getBin(hand['hp'], 8, 3)] +=1
        except KeyError:
            pass
    return handHp
def getHandDurability(state):
    handDur = np.zeros(3)
    # maxval is 4, we divide by 3
    for hand in state['player']['hand']:
        if hand['type'] == "WEAPON":
            try:
                handDur[getBin(hand['durability'], 4, 3)] +=1
            except KeyError:
                pass
    return handDur
def getHandAttackHistogram(state, type):
    # maxval is 8, we divide by 3 folds
    maxval = 8
    div = 3
    handAttack = np.zeros(div)
    if type == "WEAPON":
        # maxval is 5, we divide by 3
        maxval = 5
        handAttack = np.zeros(div)
    for hand in state['player']['hand']:
        if hand['type'] == type:
            try:
                handAttack[getBin(hand['attack'], maxval, div)] +=1
            except KeyError:
                pass
    return handAttack
def getHandCrystalCostHistogram(state, type):
    # maxval is 8
    maxval = 8
    div = 3
    handCrystalCost = np.zeros(div) # we divide to 4 folds
    
    if type == "WEAPON":
        # maxval is 5
        handCrystalCost = np.zeros(div) # we divide to 3 folds
        maxval = 5
    if type == "SPELL":
        # maxval is 7
        handCrystalCost = np.zeros(div) # we divide to 3 folds
        maxval = 7
    for hand in state['player']['hand']:
        if hand['type'] == type:
            try:
                handCrystalCost[getBin(hand['crystals_cost'], maxval, div)] +=1
            except KeyError:
                pass
    return handCrystalCost

def getHandSumBySpecific(state, attr, specific):
    sum = 0
    for hand in state['player']['hand']:
        try:
            if hand[specific] is True and hand['type'] == "MINION":
                sum+=hand[attr]
        except KeyError:
            pass
    return sum

def getPlayedCardsSumBySpecific(state, attr, specific, type):
    sum = 0
    for hand in state[type]['played_cards']:
        try:
            if hand[specific] is True:
                sum+=hand[attr]
        except KeyError:
            pass
    return sum

def getPlayedCardsAllSpecifics(state, type):
    attrs = ['hp_max', 'hp_current', 'attack', 'crystals_cost']
    specifs = ['taunt', 'charge', 'freezing', 'frozen', 'can_attack']
    specs = np.zeros(20)
    i = 0
    for specif in specifs:
        for attr in attrs:
            specs[i] = getPlayedCardsSumBySpecific(state, attr, specif, type)
            i+=1
    return specs

def getHandAllSpecifics(state):
    attrs = ['hp', 'attack', 'crystals_cost']
    specifs = ['taunt', 'charge', 'freezing']
    specs = np.zeros(9)
    i = 0
    for specif in specifs:
        for attr in attrs:
            specs[i] = getHandSumBySpecific(state, attr, specif)
            i+=1
    return specs

def getHandCountsOfTypes(state):
    counts = np.zeros(3)
    for hand in state['player']['hand']:
        try:
            if hand['type']=="MINION":
                counts[0]+=1
            elif hand['type']=="WEAPON":
                counts[1]+=1    
            elif hand['type']=="SPELL":
                counts[2]+=1    
        except KeyError:
            pass
    return counts

def getBin(val, max, nb):
    if val > max:
        val = max
    dx = max/nb
    if max % nb >0:
        dx+=1
    r = val/dx
    if val == max and max % nb == 0:
        r-=1
    return r

def getPlayerStats(state, player):
    stats = np.zeros(6)
    i = 0
    data = state[player]['stats']
    stats[0] = data['crystals_all']
    stats[1] = data['crystals_current']
    stats[2] = data['deck_count']
    stats[3] = data['fatigue_damage']
    stats[4] = data['hand_count']
    stats[5] = data['played_minions_count']
    
    return stats

def getHeroStats(state, player):
    stats = np.zeros(5)
    data = state[player]['hero']
    stats[0] = data['armor']
    stats[1] = data['attack']
    stats[2] = data['hp']
    stats[3] = data['special_skill_used'] == True
    stats[4] = data['weapon_durability']
    
    return stats

def getPlayerPlayedCardsHpCurrentHistogram(state):
    #maxval is 24, we divide to 4 folds
    handHp = np.zeros(4)
    for hand in state['player']['played_cards']:
        try:
            handHp[getBin(hand['hp_current'], 24, 4)] +=1
        except KeyError:
            pass
    return handHp
def getOpponentPlayedCardsHpCurrentHistogram(state):
    #maxval is 32, we divide to 5 folds
    handHp = np.zeros(5)
    for hand in state['opponent']['played_cards']:
        try:
            handHp[getBin(hand['hp_current'], 32, 5)] +=1
        except KeyError:
            pass
    return handHp
def getPlayerPlayedCardsHpMaxHistogram(state):
    #maxval is 30, we divide to 3 folds
    handHp = np.zeros(3)
    for hand in state['player']['played_cards']:
        try:
            handHp[getBin(hand['hp_max'], 30, 3)] +=1
        except KeyError:
            pass
    return handHp
def getOpponentPlayedCardsHpMaxHistogram(state):
    #maxval is 37, we divide to 4 folds
    handHp = np.zeros(4)
    for hand in state['opponent']['played_cards']:
        try:
            handHp[getBin(hand['hp_max'], 37, 4)] +=1
        except KeyError:
            pass
    return handHp
def getPlayerPlayedCardsAttackHistogram(state):
    #maxval is 23, we divide to 4 folds
    handHp = np.zeros(4)
    for hand in state['player']['played_cards']:
        try:
            handHp[getBin(hand['attack'], 23, 4)] +=1
        except KeyError:
            pass
    return handHp
def getOpponentPlayedCardsAttackHistogram(state):
    #maxval is 19, we divide to 3 folds
    handHp = np.zeros(3)
    for hand in state['opponent']['played_cards']:
        try:
            handHp[getBin(hand['attack'], 19, 3)] +=1
        except KeyError:
            pass
    return handHp
def getPlayedCardsCrystalsCostHistogram(state, type):
    #maxval is 8, we divide to 3 folds
    handHp = np.zeros(3)
    for hand in state[type]['played_cards']:
        try:
            handHp[getBin(hand['crystals_cost'], 8, 3)] +=1
        except KeyError:
            pass
    return handHp
def getAllMinMax(state):
    return np.hstack([getMinMaxBySpecific(state,'hp'), getMinMaxBySpecific(state,'crystals_cost'),
           getMinMaxBySpecific(state,'attack')])

def craftAllFeatures(state):
    return np.hstack([state['gamestate_id'], getHandHpHistogram(state), getHandAttackHistogram(state),
                              getHandCrystalCostHistogram(state), getAllSpecifics(state),
                              getAllMinMax(state)])

def read_from_pickle(path):
    with open(path, 'rb') as file:
        try:
            return pickle.load(file)
        except EOFError:
            pass    

def craftAllFeatures(state):
    return np.hstack([
            getHandHpHistogram(state), getHandDurability(state), getHandAttackHistogram(state, "MINION"),
            getHandAttackHistogram(state, "WEAPON"), getHandCrystalCostHistogram(state, "MINION"),
            getHandCrystalCostHistogram(state, "WEAPON"), getHandCrystalCostHistogram(state, "SPELL"),
            getHandAllSpecifics(state), getPlayedCardsAllSpecifics(state, 'player'), getHandCountsOfTypes(state),
            getPlayerStats(state, 'player'), getHeroStats(state, 'player'), 
            getPlayerPlayedCardsHpCurrentHistogram(state), getPlayerPlayedCardsHpMaxHistogram(state),
            getPlayerPlayedCardsAttackHistogram(state), getPlayedCardsCrystalsCostHistogram(state, 'player'),
            getOpponentPlayedCardsHpCurrentHistogram(state), getOpponentPlayedCardsHpMaxHistogram(state),
            getOpponentPlayedCardsAttackHistogram(state), getPlayedCardsCrystalsCostHistogram(state, 'opponent'),
            getPlayedCardsAllSpecifics(state, 'opponent')
        ])

def getAllJsonTraining(data, id):
    handsHist = np.empty((0, 113), float)
    for state in data:
        handsHist = np.vstack([handsHist, craftAllFeatures(state)])
    pickle.dump(handsHist, open(dataFeaturesOutput+'handsHist-%d.store'%id,"wb"))

i = 5
trainChunk = read_from_pickle(dataFeaturesOutput+'featureChunk-%d.store'%i)
getAllJsonFeatures(trainChunk, i)
    
    
# for random feature selection
dataFeaturesPrefix = dataPrefix+"features/"
filepathToModels = dataPrefix+'dims-heavy-79.store'

def stripDim(data, indexes):
    indexes.sort(reverse=True)
    for i in indexes:
        data = np.concatenate((data[:,0:i], data[:,i+1:len(data[1,:])]), axis = 1)
    return data
def takeSampleDim(train):
    dim = train.shape[1]
    range2strip = dim/5
    nb2strip = randint(1, range2strip)
    return random.sample(range(1, dim), nb2strip)
def persistSample(train, sample, sampleSet):
    while frozenset(sample) in sampleSet:
        sample = takeSampleDim(train)
    sampleSet.add(frozenset(sample))
    pickle.dump(sampleSet, open(filepathToModels,"wb"))
    return sample
def stripDimSets(X_train, X_test, dim2strip):
    X_train_str = stripDim(X_train, dim2strip)
    X_test_str  = stripDim(X_test, dim2strip)
    return (X_train_str, X_test_str)
def trainClassifier(clf, X_train, y_train, X_test, y_test):
    clf = clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    return metrics.roc_auc_score(y_test, predicted)
def writeSolution(sample, auc):
    filepath = dataPrefix+"chosen-heavy-79.store"
    if os.path.isfile(filepath):
        sampleF, aucF = read_from_pickle(filepath)
        if sampleF is not None:
            if auc > float(aucF):
                pickle.dump((sample, auc), open(filepath,"wb"))
    else:
        pickle.dump((sample, auc), open(filepath,"wb"))
def makeOneStep(clf, filepathToModels, data):
    xtrain, ytrain, xtest, ytest = data
    sampleSet = read_from_pickle(filepathToModels)
    if sampleSet is None:
        sampleSet = set()
    sample = takeSampleDim(xtrain)
    sample = persistSample(xtrain, sample, sampleSet)
    xtrain, xtest = stripDimSets(xtrain, xtest, sample)
    auc = trainClassifier(clf, xtrain, ytrain, xtest, ytest)
    writeSolution(sample, auc)
    return (auc, sample)


# Yet another Feature Computation

In [ ]:
import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import (RandomTreesEmbedding, IsolationForest, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder    
import json

def replaceNan(data, item):
    return [x if x is not None else item for x in data]

def getDenseFeatures(dataFrame, names, dictionary):
    X_dense = np.empty((len(dataFrame), 0), float)
    for name in names:
        values = dataFrame[name].values
        X_dense = np.column_stack([X_dense, replaceNan([dictionary[name].get(value) for value in values], 0.5)])
    return X_dense
def addNewFeature(data, vectors):
    return np.column_stack([data, vectors])
def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature
def dataScaling(data, scaler):
    data = data/(data+1)
    return scaler.fit_transform(data)

def _computeFractionForAttribute(dataFrame, name):
    dictionary = dict()
    values = set(dataFrame[name].values)
    for value in values:
        ind = dataFrame[name].values==value
        subset = dataFrame['decision'][ind].values
        fraction = 1.0*len(filter(lambda x: x==1, subset)) / len(subset)
        dictionary[value] = fraction 
    return dictionary
def computeFractionsForAll(dataFrame, names):
    changeDict = dict()
    for name in names:
        changeDict[name] = _computeFractionForAttribute(dataFrame, name)
    return changeDict
def dropColumnsFromDataFrame(df, columns):
    for column in columns:
        df = df.drop(column, 1)
    return df

def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
def printify(t):
    for x in t:
        print(x)
def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)

def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)

dataPrefix = '/home/data/aaia17/'
dataDepreciatedPrefix = dataPrefix+"depreciated/"
def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
def printify(t):
    for x in t:
        print(x)
def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)    
def readJsonData(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        with open(fData, 'r') as f:
            frame.append(json.load(f))
        f.close()
    return frame  
def readJsonDatabyLine(prefix, fNames):
    frame = []
    for fileName in fNames:
        fData = prefix+fileName
        frame.extend(readFileByLine(fData))
    return frame
def readJsonFromString(data):
    result = []
    nulls = []
    for i in range(0,len(data)):
        result.append(json.loads(data[i]))
                    #nulls.append(i)
    return result
def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature

def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)
def readFileByLine(fileName):
    arr = []
    with open(fileName, 'r') as f:        
        for line in f:
            arr.append(line.rstrip())
        f.close()
    return arr

def getHandHpHistogram(state):
    handHp = np.zeros(3)
    # maxval is 8, we divide by 3
    for hand in state['player']['hand']:
        try:
            handHp[getBin(hand['hp'], 8, 3)] +=1
        except KeyError:
            pass
    return handHp
def getHandDurability(state):
    handDur = np.zeros(3)
    # maxval is 4, we divide by 3
    for hand in state['player']['hand']:
        if hand['type'] == "WEAPON":
            try:
                handDur[getBin(hand['durability'], 4, 3)] +=1
            except KeyError:
                pass
    return handDur
def getHandAttackHistogram(state, type):
    # maxval is 8, we divide by 3 folds
    maxval = 8
    div = 3
    handAttack = np.zeros(div)
    if type == "WEAPON":
        # maxval is 5, we divide by 3
        maxval = 5
        handAttack = np.zeros(div)
    for hand in state['player']['hand']:
        if hand['type'] == type:
            try:
                handAttack[getBin(hand['attack'], maxval, div)] +=1
            except KeyError:
                pass
    return handAttack
def getHandCrystalCostHistogram(state, type):
    # maxval is 8
    maxval = 8
    div = 3
    handCrystalCost = np.zeros(div) # we divide to 4 folds
    
    if type == "WEAPON":
        # maxval is 5
        handCrystalCost = np.zeros(div) # we divide to 3 folds
        maxval = 5
    if type == "SPELL":
        # maxval is 7
        handCrystalCost = np.zeros(div) # we divide to 3 folds
        maxval = 7
    for hand in state['player']['hand']:
        if hand['type'] == type:
            try:
                handCrystalCost[getBin(hand['crystals_cost'], maxval, div)] +=1
            except KeyError:
                pass
    return handCrystalCost

def getHandSumBySpecific(state, attr, specific):
    sum = 0
    for hand in state['player']['hand']:
        try:
            if hand[specific] is True and hand['type'] == "MINION":
                sum+=hand[attr]
        except KeyError:
            pass
    return sum

def getPlayedCardsSumBySpecific(state, attr, specific, type):
    sum = 0
    for hand in state[type]['played_cards']:
        try:
            if hand[specific] is True:
                sum+=hand[attr]
        except KeyError:
            pass
    return sum

def getPlayedCardsAllSpecifics(state, type):
    attrs = ['hp_max', 'hp_current', 'attack', 'crystals_cost']
    specifs = ['taunt', 'charge', 'freezing', 'frozen', 'can_attack']
    specs = np.zeros(20)
    i = 0
    for specif in specifs:
        for attr in attrs:
            specs[i] = getPlayedCardsSumBySpecific(state, attr, specif, type)
            i+=1
    return specs

def getHandAllSpecifics(state):
    attrs = ['hp', 'attack', 'crystals_cost']
    specifs = ['taunt', 'charge', 'freezing']
    specs = np.zeros(9)
    i = 0
    for specif in specifs:
        for attr in attrs:
            specs[i] = getHandSumBySpecific(state, attr, specif)
            i+=1
    return specs

def getHandCountsOfTypes(state):
    counts = np.zeros(3)
    for hand in state['player']['hand']:
        try:
            if hand['type']=="MINION":
                counts[0]+=1
            elif hand['type']=="WEAPON":
                counts[1]+=1    
            elif hand['type']=="SPELL":
                counts[2]+=1    
        except KeyError:
            pass
    return counts

def getBin(val, max, nb):
    if val > max:
        val = max
    dx = max/nb
    if max % nb >0:
        dx+=1
    r = val/dx
    if val == max and max % nb == 0:
        r-=1
    return r

def getPlayerStats(state, player):
    stats = np.zeros(6)
    i = 0
    data = state[player]['stats']
    stats[0] = data['crystals_all']
    stats[1] = data['crystals_current']
    stats[2] = data['deck_count']
    stats[3] = data['fatigue_damage']
    stats[4] = data['hand_count']
    stats[5] = data['played_minions_count']
    
    return stats

def getHeroStats(state, player):
    stats = np.zeros(5)
    data = state[player]['hero']
    stats[0] = data['armor']
    stats[1] = data['attack']
    stats[2] = data['hp']
    stats[3] = data['special_skill_used'] == True
    stats[4] = data['weapon_durability']
    
    return stats

def getPlayerPlayedCardsHpCurrentHistogram(state):
    #maxval is 24, we divide to 4 folds
    handHp = np.zeros(4)
    for hand in state['player']['played_cards']:
        try:
            handHp[getBin(hand['hp_current'], 24, 4)] +=1
        except KeyError:
            pass
    return handHp
def getOpponentPlayedCardsHpCurrentHistogram(state):
    #maxval is 32, we divide to 5 folds
    handHp = np.zeros(5)
    for hand in state['opponent']['played_cards']:
        try:
            handHp[getBin(hand['hp_current'], 32, 5)] +=1
        except KeyError:
            pass
    return handHp
def getPlayerPlayedCardsHpMaxHistogram(state):
    #maxval is 30, we divide to 3 folds
    handHp = np.zeros(3)
    for hand in state['player']['played_cards']:
        try:
            handHp[getBin(hand['hp_max'], 30, 3)] +=1
        except KeyError:
            pass
    return handHp
def getOpponentPlayedCardsHpMaxHistogram(state):
    #maxval is 37, we divide to 4 folds
    handHp = np.zeros(4)
    for hand in state['opponent']['played_cards']:
        try:
            handHp[getBin(hand['hp_max'], 37, 4)] +=1
        except KeyError:
            pass
    return handHp
def getPlayerPlayedCardsAttackHistogram(state):
    #maxval is 23, we divide to 4 folds
    handHp = np.zeros(4)
    for hand in state['player']['played_cards']:
        try:
            handHp[getBin(hand['attack'], 23, 4)] +=1
        except KeyError:
            pass
    return handHp
def getOpponentPlayedCardsAttackHistogram(state):
    #maxval is 19, we divide to 3 folds
    handHp = np.zeros(3)
    for hand in state['opponent']['played_cards']:
        try:
            handHp[getBin(hand['attack'], 19, 3)] +=1
        except KeyError:
            pass
    return handHp
def getPlayedCardsCrystalsCostHistogram(state, type):
    #maxval is 8, we divide to 3 folds
    handHp = np.zeros(3)
    for hand in state[type]['played_cards']:
        try:
            handHp[getBin(hand['crystals_cost'], 8, 3)] +=1
        except KeyError:
            pass
    return handHp

def read_from_pickle(path):
    with open(path, 'rb') as file:
        try:
            return pickle.load(file)
        except EOFError:
            pass    

def craftAllFeatures(state):
    return np.hstack([
            getHandHpHistogram(state), getHandDurability(state), getHandAttackHistogram(state, "MINION"),
            getHandAttackHistogram(state, "WEAPON"), getHandCrystalCostHistogram(state, "MINION"),
            getHandCrystalCostHistogram(state, "WEAPON"), getHandCrystalCostHistogram(state, "SPELL"),
            getHandAllSpecifics(state), getPlayedCardsAllSpecifics(state, 'player'), getHandCountsOfTypes(state),
            getPlayerStats(state, 'player'), getHeroStats(state, 'player'), 
            getPlayerPlayedCardsHpCurrentHistogram(state), getPlayerPlayedCardsHpMaxHistogram(state),
            getPlayerPlayedCardsAttackHistogram(state), getPlayedCardsCrystalsCostHistogram(state, 'player'),
            getOpponentPlayedCardsHpCurrentHistogram(state), getOpponentPlayedCardsHpMaxHistogram(state),
            getOpponentPlayedCardsAttackHistogram(state), getPlayedCardsCrystalsCostHistogram(state, 'opponent'),
            getPlayedCardsAllSpecifics(state, 'opponent')
        ])

dataFeaturesOutput = '/home/data/aaia17/features/'

def getAllJsonTraining(data, id):
    handsHist = np.empty((0, 113), float)
    for state in data:
        handsHist = np.vstack([handsHist, craftAllFeatures(state)])
    pickle.dump(handsHist, open(dataFeaturesOutput+'handsHist-%d.store'%id,"wb"))
    #return handsHist

i = 0
#hunk = read_from_pickle(dataFeaturesOutput+'featureChunk-%d.store'%i)
#getAllJsonTraining(chunk, i)                         

In [ ]:
trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
testFiles = ['testData_tabular.csv']
rawTrainingDataset = readDataAsFrame(trainingFiles)
rawTestDataset = readDataAsFrame(testFiles)

exclude_features = ['opponent.hero_card_id', 'player.hero_card_id']
rawTrainingDataset = rawTrainingDataset.drop(exclude_features, axis = 1)
rawTestDataset = rawTestDataset.drop(exclude_features, axis = 1)

YRawTrainingDataset = rawTrainingDataset.values[:,1]
XRawTrainingDataset = rawTrainingDataset.values[:,2:43]
XRawTestDataset = rawTestDataset.values[:,2:43]

min_max_scaler = preprocessing.MinMaxScaler()
XTrainingDatasetScaled = dataScaling(XRawTrainingDataset, min_max_scaler)
XTestDatasetScaled = dataScaling(XRawTestDataset, min_max_scaler)

dtrain = readJsonDatabyLine(dataPrefix, ["trainingData_JSON_chunk1.json", "trainingData_JSON_chunk2.json", 
                                                "trainingData_JSON_chunk3.json", "trainingData_JSON_chunk4.json"])
jtrain = readJsonFromString(dtrain)
dtest = readJsonDatabyLine(dataDepreciatedPrefix, ["testData_JSON_chunk5.json", "testData_JSON_chunk6.json", 
                                                "testData_JSON_chunk7.json"])
jtest = readJsonFromString(dtest)
ftrain = getAllJsonFeatures(jtrain)
fest = getAllJsonFeatures(jtest)

i = 0
for state in jtrain[:100]:
    for hand in state['player']['hand']:
        try:
            if hand['type'] == u'SPELL':
                    print hand
                    i+=1
        except:
            pass
    if i==10:
        break
state = jtrain[0]
np.hstack([getHandHpHistogram(state), getHandAttackHistogram(state, "MINION"), 
                              getHandCrystalCostHistogram(state, "MINION"), getAllSpecifics(state),
                              getHandAttackHistogram(state, "WEAPON"), getHandCrystalCostHistogram(state, "WEAPON"),
                              getHandDurability(state), getHandCrystalCostHistogram(state, "SPELL")]).shape        

In [ ]:
dataTrain = np.empty((0, 109), int)
for i in range(0,10):
    dataTrain = np.vstack([dataTrain, read_from_pickle(dataFeaturesPrefix+'handsHist-exp-%d.store'%i)])
dataTrain = dataTrain[:,1:]    

dataTest = np.empty((0, 109), int)
for i in range(0,5):
    dataTest = np.vstack([dataTest, read_from_pickle(dataFeaturesPrefix+'handsHistTest-exp2-%d.store'%i)])
dataTest = dataTest[:,1:]       

np.save(dataFeaturesOutput+"trainSetScaled-old.dat", XTrainingDatasetScaledWithOutliersNew)
np.save(dataFeaturesOutput+"testSetScaled-old.dat", XTestDatasetScaledNew)

r = np.zeros(750000)
for i in range(0,750000):
    r[i] = np.all(XRawTest[i,:25] == test_sparse[i,:25])

np.all(r == True)

In [ ]:
trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
trainingFilesDepr = ['testData_tabular_chunk5.csv', 'testData_tabular_chunk6.csv',
                 'testData_tabular_chunk7.csv']
testFiles = ['testData_tabular.csv']
rawTraining = readDataAsFrame(dataPrefix, trainingFiles)
rawTest     = readDataAsFrame(dataPrefix, testFiles)

rawTrainingDepr = readDataAsFrame(dataDepreciatedPrefix, trainingFilesDepr)
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

exclude_features = ['opponent.hero_card_id', 'player.hero_card_id']
rawTrainingDataset = rawTraining.drop(exclude_features, axis = 1)
rawTrainingDeprDataset = rawTrainingDepr.drop(exclude_features, axis = 1)
rawTestDataset = rawTest.drop(exclude_features, axis = 1)

XRawTrain = rawTrainingDataset.values[:,2:43]
XRawTrainDepr = rawTrainingDeprDataset.values[:,2:43]
XRawTest_  = rawTestDataset.values[:,2:43]

XRawTrainAll_ = np.vstack([XRawTrain, XRawTrainDepr])
XRawTrainAll  = np.hstack([XRawTrainAll_, dataTrain])
XRawTest      = np.hstack([XRawTest_, dataTest])

min_max_scaler = preprocessing.MinMaxScaler()
XTrain_ = dataScaling(XRawTrainAll, min_max_scaler)
XTest_  = dataScaling(XRawTest, min_max_scaler)

labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

dimToRemove = getDimToRemove(XTrain_, XTest_)
XTrain = stripDim(XTrain_, dimToRemove)
XTest  = stripDim(XTest_, dimToRemove)

dataFeaturesOutput = '/home/data/aaia17/features/'
np.save(dataFeaturesOutput+"train-features-146-bins.dat", XTrain)
np.save(dataFeaturesOutput+"test-features-146-bins.dat", XTest)

# PCA

In [ ]:
pca = decomposition.PCA(svd_solver='randomized', iterated_power=7, n_components=63)
pca.fit(XTraining)
min_max_scaler = preprocessing.MinMaxScaler()
XTrainingPCA = pca.transform(XTraining) 
XTestPCA = pca.transform(XTest) 
XTrainingPCA = dataScaling(XTrainingPCA, min_max_scaler)
XTestPCA = dataScaling(XTestPCA, min_max_scaler)
pickle.dump(XTrainingPCA, open(dataPrefix+'XTraining63PCA.store',"wb"))
pickle.dump(XTestPCA, open(dataPrefix+'XTest63PCA.store',"wb"))
print(pca.explained_variance_ratio_) 

# Data reading from all chunks and making one  dataset

In [161]:
trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
rawTrainingDataset = readDataAsFrame(trainingFiles)

testFiles = ['testData_tabular.csv']
rawTestDataset = readDataAsFrame(testFiles)

In [162]:
print(rawTrainingDataset.groupby('decision').size())

decision
0     990401
1    1009599
dtype: int64


In [ ]:
rawTrainingDataset[]

# Take All Features

In [163]:
XRawTrainingDataset = rawTrainingDataset.values[:,2:45]
YRawTrainingDataset = rawTrainingDataset.values[:,1]
XRawTestDataset = rawTestDataset.values[:,2:45]

# Hand chosen features

In [7]:
# Actually, we don't use it !

names = ['opponent.hp', 'opponent.played_minions_count', 'player.armor','player.hp', 'player.hand_count', 
         'player.played_minions_count', 'opponent.played.nOfCards', 'opponent.played.attack', 'opponent.played.crystals_cost'
         ,'opponent.played.hp_current', 'opponent.played.hp_max', 'player.played.nOfCards', 'player.played.attack',
        'player.played.crystals_cost', 'player.played.hp_current', 'player.played.hp_max', 'player.hand.nOfCards']
X_train_handChosen = getNewForIndexes(rawTrainingDataset, names)
X_test_handChosen  = getNewForIndexes(rawTestDataset, names)

#  Outliers detection and removing

In [ ]:
%%time
# Actually, we don't use it !

YRawTrainingDataset = rawTrainingDataset.values[:,1]
XRawTrainingDataset = rawTrainingDataset.values[:,2:45]

rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=10000, random_state=rng, n_jobs=-1)
clf.fit(XRawTrainingDataset)

trainingOutliers = clf.predict(XRawTrainingDataset)

XTrainigDatasetWithoutOutliers = XRawTrainingDataset[trainingOutliers>0]
YTrainingDatasetWithoutOutliers = YRawTrainingDataset[trainingOutliers>0]

# Automatic feature selection 

In [ ]:
%%time
# Actually, we don't use it !

estimator = LogisticRegression(random_state=rng)
selector = RFECV(estimator, step=1, cv=10, scoring='roc_auc', n_jobs=-1)
XTrainingSelected = selector.fit_transform(Xtry, Ytry)
pickle.dump(XTrainingSelected, open(dataPrefix+'XTrainingSelected.store',"wb"))

selector.ranking_

# Data scaling

In [8]:
min_max_scaler = preprocessing.MinMaxScaler()

XTrainingDatasetScaledWithOutliers = dataScaling(X_train_handChosen, min_max_scaler)
XTestDatasetScaled = dataScaling(X_test_handChosen, min_max_scaler)

# Creating train and validation probes

In [ ]:
validation_size = 0.30
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(XTrainingDatasetScaledWithOutliers, 
        YRawTrainingDataset, test_size=validation_size, random_state=seed)

# Models probing

In [ ]:
# Actually, we don't use it !
%%time

seed = 7
scoring = 'roc_auc'
models = []
models.append(('RFC', RandomForestClassifier(n_jobs=-1, n_estimators=40)))

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    
# Prediction on Test Set
validation_size = 0.30
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(XTrainingDatasetScaledWithOutliers, 
        YRawTrainingDataset, test_size=validation_size, random_state=seed)
predicted = clf.predict(X_validation)
predictedTestWon = predicted[:,1]
print(metrics.classification_report(Y_validation, predicted))

# Grid Search

In [ ]:
param_test1 = {'n_estimators':range(20,101,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=1.9503, min_samples_split=500,min_samples_leaf=50,max_depth=4,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring='roc_auc',n_jobs=40,iid=False, cv=5)
gsearch1.fit(XTraining, YTraining)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

param_test2 = {'max_depth':range(4,12,2), 'min_samples_split':range(100,1101,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=1.9503, min_samples_leaf=50,
                                                               max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test2, scoring='roc_auc',n_jobs=40,iid=False, cv=5)
gsearch2.fit(XTraining, YTraining)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

# Averaging

In [ ]:
allFiles = [f for f in os.listdir(averageDir)]
mean = meanVector(allFiles)
writeToFile(mean, dataPrefix+'final.txt')    
# with weights
weightsHand = [0.7961,0.7954,0.7954,0.7954,0.7954,0.7953,0.7953,0.7951,0.795,0.795,0.795,0.7949,0.7946,0.7944,0.7944,0.7943,0.7942,0.794,0.794,0.7939,0.7939,0.7938,0.7938,0.7938,0.7936,0.7936,0.7936,0.7935,0.7935,0.7935,0.7935,0.7934,0.7934,0.7933,0.793,0.7929]
files=[1000,78,74,66,55,92,51,59,63,96,52,95,62,77,61,84,65,81,80,64,91,82,88,93,37,86,39,58,89,53,34,40,85,38,87,79]
meanPredictedW = meanVectorW("predicted", files[0:17], weightsHand[0:17])
writeToFile(meanPredictedW, dataPrefix+'mean17.txt')

# Voting

In [ ]:
allFiles = [f for f in os.listdir(modelsDir)]
models = []
modelsList = []
for f in allFiles:
    models.append((f, read_from_pickle(modelsDir+f)))
    modelsList.append(read_from_pickle(modelsDir+f))

map(lambda x: x.n_features, modelsList)    

eclf = VotingClassifier(estimators=models, voting='hard', n_jobs=40)
eclf = eclf.fit(XTraining, YTraining)

eclf = VotingClassifier(estimators=models, voting='soft', n_jobs=20)
eclf.estimators_ = modelsList
#eclf = eclf.fit(XTraining, YTraining)
predictedTestWon = eclf.predict_proba(XTest)[:,1]
writeToFile(predictedTestWon, dataPrefix+'voting4.txt')

# Staging

In [ ]:
XTrain = read_sparse_binary_set(dataFeaturesPrefix + 'train-features-79-old-approach.dat.npy')[:,:-6]
XTest  = read_sparse_binary_set(dataFeaturesPrefix + 'test-features-79-old-approach.dat.npy')[:,:-6]
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')


X_train, X_test, y_train, y_test = train_test_split(XTrain, labels, test_size=0.4, random_state=0)

print XTrain.shape, XTest.shape

i = 1
est = 450
md  = 5
mss = 50
msl = 5


clf = GradientBoostingClassifier(n_estimators=est, max_depth=md, min_samples_split = mss, min_samples_leaf = msl, warm_start=True)
clf = clf.fit(X_train, y_train)
#predictedTestWon = clf.predict_proba(XTest)[:,1]
#writeToFile(predictedTestWon, dataPrefix+'fb%d.txt'%i)
pickle.dump(clf, open(dataPrefix+'overfit-gbc-old-6-%d.model'%i,"wb"))


test_score = []
train_score = []
for i, pred in enumerate(clf.staged_decision_function(X_test)):
    test_score.append((i,metrics.roc_auc_score(y_test, pred)))
for i, pred in enumerate(clf.staged_decision_function(X_train)):
    train_score.append((i, metrics.roc_auc_score(y_train, pred)))
plt.plot(*zip(*test_score))
plt.plot(*zip(*train_score))
plt.legend(['test score', 'train score'])

# Random Feature Selection

In [ ]:
XTrain = read_sparse_binary_set(dataFeaturesPrefix + 'train-features-79-old-approach.dat.npy')
XTest  = read_sparse_binary_set(dataFeaturesPrefix + 'test-features-79-old-approach.dat.npy')
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

X_train, X_test, y_train, y_test = train_test_split(XTrain, labels, test_size=0.4, random_state=0)
data = (X_train, y_train, X_test, y_test)
clf = GradientBoostingClassifier(n_estimators=150, max_depth=4, min_samples_split = 50, min_samples_leaf = 5)

for i in range(0,1000):
    r = makeOneStep(clf, filepathToModels, data)
    print i, r


# Gaussian Mixture Model

In [ ]:
allFiles = [f for f in os.listdir(averageDir)]
models = map(lambda x: converseVec2Matrix(readFileByLine(averageDir+x)), allFiles)
X_train = np.hstack(models)
f1 = converseVec2Matrix(readFileByLine(averageDir+"predicted78.txt"))
f2 = converseVec2Matrix(readFileByLine(averageDir+"predicted74.txt"))
f3 = converseVec2Matrix(readFileByLine(averageDir+"predicted66.txt"))
f4 = converseVec2Matrix(readFileByLine(averageDir+"predicted55.txt"))
f5 = converseVec2Matrix(readFileByLine(averageDir+"predicted1000.txt"))
f6 = converseVec2Matrix(readFileByLine(dataPrefix+"mean15.txt"))
f7 = converseVec2Matrix(readFileByLine(dataPrefix+"mean12.txt"))
f8 = converseVec2Matrix(readFileByLine(dataPrefix+"mean14.txt"))

X_train = np.hstack([f1, f2, f3, f4, f5, f6, f7, f8])

clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
clf.fit(X_train)

arr = np.empty((len(preds), 1), float)
for i in range(0, len(preds)):
    arr[i] = np.dot(preds[i, :], X_train[i, :]) # ?????????????????

# Model Training

In [ ]:
XTrain = read_sparse_binary_set(dataFeaturesPrefix + 'train-features-79-old-approach.dat.npy')[:-1250000]
XTest  = read_sparse_binary_set(dataFeaturesPrefix + 'test-features-79-old-approach.dat.npy')
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')[:-1250000]

print XTrain.shape, XTest.shape, labels.shape

i = 1
est = 150
md  = 5
mss = 50
msl = 5

clf = GradientBoostingClassifier(n_estimators=est, max_depth=md, min_samples_split = mss, min_samples_leaf = msl)
clf = clf.fit(XTrain, labels)
predictedTestWon = clf.predict_proba(XTest)[:,1]
writeToFile(predictedTestWon, dataPrefix+'f-1mln%d.txt'%i)
pickle.dump(clf, open(dataPrefix+'gbc-f79-1mln-%d.model'%i,"wb"))

clf.set_params(n_estimators=130, warm_start=True)

In [ ]:
XTrain = read_sparse_binary_set(dataFeaturesPrefix + 'train-features-64-bins.dat.npy')
XTest  = read_sparse_binary_set(dataFeaturesPrefix + 'test-features-64-bins.dat.npy')
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

i = 6
est = 100
md  = 5
mss = 100
msl = 10

clf = GradientBoostingClassifier(n_estimators=est, max_depth=md, min_samples_split = mss, min_samples_leaf = msl, warm_start=True)
clf = clf.fit(XTrain, labels)
predictedTestWon = clf.predict_proba(XTest)[:,1]
writeToFile(predictedTestWon, dataPrefix+'fb%d.txt'%i)
pickle.dump(clf, open(dataPrefix+'gbc-bins-64-%d.model'%i,"wb"))


In [ ]:
sample = XTrain = read_sparse_binary_set(dataFeaturesPrefix + 'train-features-79-old-approach.dat.npy')[:,:-6]
XTest  = read_sparse_binary_set(dataFeaturesPrefix + 'test-features-79-old-approach.dat.npy')[:,:-6]
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

print XTrain.shape, XTest.shape

sample = [71, 70, 59, 58, 32, 27, 22, 19, 17, 13, 6]
xtrain, xtest = stripDimSets(XTrain, XTest, sample)
print xtrain.shape, xtest.shape


i = 10
est = 150
md  = 5
mss = 100
msl = 10

clf = GradientBoostingClassifier(n_estimators=est, max_depth=md, min_samples_split = mss, min_samples_leaf = msl, warm_start=True)
clf = clf.fit(xtrain, labels)
predictedTestWon = clf.predict_proba(xtest)[:,1]
writeToFile(predictedTestWon, dataPrefix+'f79-6-s%d.txt'%i)
pickle.dump(clf, open(dataPrefix+'gbc-old-79-6-s%d.model'%i,"wb"))

# Neural Networks

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
from sklearn import model_selection
import pickle

def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)

def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()

dataPrefix = '/home/data/aaia17/'
dataFeaturesPrefix = dataPrefix+"features/"

XTrain = read_sparse_binary_set(dataFeaturesPrefix + 'train-features-79-old-approach.dat.npy')
XTest  = read_sparse_binary_set(dataFeaturesPrefix + 'test-features-79-old-approach.dat.npy')
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

labelsKeras = keras.utils.to_categorical(labels, num_classes=2)

def initializeNN(frameSize):
    model = Sequential()
    model.add(Dense(units=81, input_dim=frameSize, activation=K.tanh))
    model.add(Activation(K.tanh))
    model.add(Dropout(0.5))

    model.add(Dense(units=60, activation=K.tanh))
    model.add(Activation(K.tanh))
    model.add(Dropout(0.5))

    model.add(Dense(units=30, activation=K.tanh))
    model.add(Activation(K.tanh))
    model.add(Dropout(0.5))

    model.add(Dense(units=2, activation=K.softmax))
    model.add(Activation(K.softmax))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model


model = initializeNN(XTrain.shape[1])

model.fit(XTrain, labelsKeras, nb_epoch=200, verbose=1, callbacks=[], validation_data=None, shuffle=True,
          class_weight=None,
          sample_weight=None)

i = 5

y_score = model.predict(XTest)
predictedTestWon = y_score[:,1]
writeToFile(predictedTestWon, dataPrefix+'nn%d.txt'%i)
pickle.dump(model, open(dataPrefix+'n110-%d.model'%i,"wb"))

# Dump

In [ ]:
import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import (RandomTreesEmbedding, IsolationForest,VotingClassifier, RandomForestClassifier, GradientBoostingClassifier)

def replaceNan(data, item):
    return [x if x is not None else item for x in data]

def getDenseFeatures(dataFrame, names, dictionary):
    X_dense = np.empty((len(dataFrame), 0), float)
    for name in names:
        values = dataFrame[name].values
        X_dense = np.column_stack([X_dense, replaceNan([dictionary[name].get(value) for value in values], 0.5)])
    return X_dense
def addNewFeature(data, vectors):
    return np.column_stack([data, vectors])
def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature
def dataScaling(data, scaler):
    data = data/(data+1)
    return scaler.fit_transform(data)

def _computeFractionForAttribute(dataFrame, name):
    dictionary = dict()
    values = set(dataFrame[name].values)
    for value in values:
        ind = dataFrame[name].values==value
        subset = dataFrame['decision'][ind].values
        fraction = 1.0*len(filter(lambda x: x==1, subset)) / len(subset)
        dictionary[value] = fraction 
    return dictionary
def computeFractionsForAll(dataFrame, names):
    changeDict = dict()
    for name in names:
        changeDict[name] = _computeFractionForAttribute(dataFrame, name)
    return changeDict
def dropColumnsFromDataFrame(df, columns):
    for column in columns:
        df = df.drop(column, 1)
    return df

def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
def printify(t):
    for x in t:
        print(x)
dataPrefix = '/home/data/aaia17/'
def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)

def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)

def dataScaling(data, scaler):
    return scaler.fit_transform(data)

def stripDim(data, indexes):
    indexes.sort(reverse=True)
    for i in indexes:
        data = np.concatenate((data[:,0:i], data[:,i+1:len(data[1,:])]), axis = 1)
    return data

def getDimToRemove(data1, data2):
    dimToRemove = []
    for i in range(0,len(data1[0,:])):
        if np.max(data1[:,i]) == np.max(data2[:,i]) and np.max(data2[:,i]) == 0:
            dimToRemove.append(i)
    return dimToRemove

trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
rawTrainingDataset = readDataAsFrame(trainingFiles)
testFiles = ['testData_tabular.csv']
rawTestDataset = readDataAsFrame(testFiles)

training_sparse = read_sparse_binary_set(dataPrefix + '/m.npy')
test_sparse = read_sparse_binary_set(dataPrefix + '/m_test.npy')

training_sparse_New = read_sparse_binary_set(dataPrefix + 'new/m_training.npy')
test_sparse_New = read_sparse_binary_set(dataPrefix + 'new/m_test.npy')
# dense features
columnsToDensify = ['opponent.hero_card_id', 'player.hero_card_id']
chd = computeFractionsForAll(rawTrainingDataset, columnsToDensify)
X_train_denseFeatures = getDenseFeatures(rawTrainingDataset, columnsToDensify, chd)
X_test_denseFeatures  = getDenseFeatures(rawTestDataset, columnsToDensify, chd)
# drop categorial or unimportant
YRawTrainingDataset = rawTrainingDataset.values[:,1]
columnsToDrop = ['decision','gamestate_id', 'opponent.hero_card_id', 'player.hero_card_id']
rawDroppedTrainingDataset = dropColumnsFromDataFrame(rawTrainingDataset, columnsToDrop)
rawDroppedTestDataset = dropColumnsFromDataFrame(rawTestDataset, columnsToDrop)
# add some features before scaling
XRawTrainingDataset = rawDroppedTrainingDataset.values.astype(float)
XRawTestDataset     = rawDroppedTestDataset.values.astype(float)
XRawTrainingDataset = np.concatenate((XRawTrainingDataset, training_sparse[:,0:78]), axis = 1)
XRawTestDataset = np.concatenate((XRawTestDataset, test_sparse[:,0:78]), axis = 1)
XRawTrainingDataset = np.concatenate((XRawTrainingDataset, training_sparse_New[:,1:40]), axis = 1)
XRawTestDataset = np.concatenate((XRawTestDataset, test_sparse_New[:,1:40]), axis = 1)
assert np.all(training_sparse[:,78] == rawTrainingDataset.values[:,0]) and  np.all(test_sparse[:,78] == rawTestDataset.values[:,0])
assert np.all(training_sparse_New[:,0] == rawTrainingDataset.values[:,0]) and  np.all(test_sparse_New[:,0] == rawTestDataset.values[:,0])
# data scaling
min_max_scaler = preprocessing.MinMaxScaler()
XTrainingDatasetScaledWithOutliers = dataScaling(XRawTrainingDataset, min_max_scaler)
XTestDatasetScaled = dataScaling(XRawTestDataset, min_max_scaler)


clf = GradientBoostingClassifier(n_estimators=500, max_depth=5, min_samples_split = 100, min_samples_leaf = 10)
clf = clf.fit(XTrainingDatasetScaledWithOutliersNew, YRawTrainingDataset)
predictedTestWon = clf.predict_proba(XTestDatasetScaledNew)[:,1]
writeToFile(predictedTestWon, dataPrefix+'predicted67.txt')
# add new features after scaling
#XTrainingDatasetScaledWithOutliers = addNewFeature(XTrainingDatasetScaledWithOutliers, X_train_denseFeatures)
#XTestDatasetScaled = addNewFeature(XTestDatasetScaled, X_test_denseFeatures)



import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import (RandomTreesEmbedding, IsolationForest, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import json

dataPrefix = '/home/data/aaia17/'

def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()

def printify(t):
    for x in t:
        print(x)

def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)

def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature

def dataScaling(data, scaler):
    data = data/(data+1)
    return scaler.fit_transform(data)

def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)

def stripDim(data, indexes):
    indexes.sort(reverse=True)
    for i in indexes:
        data = np.concatenate((data[:,0:i], data[:,i+1:len(data[1,:])]), axis = 1)
    return data

def getDimToRemove(data1, data2):
    dimToRemove = []
    for i in range(0,len(data1[0,:])):
        if np.max(data1[:,i]) == np.max(data2[:,i]) and np.max(data2[:,i]) == 0:
            dimToRemove.append(i)
    return dimToRemove

exclude_features = ['opponent.hero_card_id', 'player.hero_card_id']

trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
rawTrainingDataset = readDataAsFrame(trainingFiles)
#rawTrainingDataset = rawTrainingDataset.drop(exclude_features, axis = 1)

testFiles = ['testData_tabular.csv']
rawTestDataset = readDataAsFrame(testFiles)
#rawTestDataset = rawTestDataset.drop(exclude_features, axis = 1)

YRawTrainingDataset = rawTrainingDataset.values[:,1]
XRawTrainingDataset = rawTrainingDataset.values[:,2:45]
XRawTestDataset = rawTestDataset.values[:,2:45]

training_sparse = read_sparse_binary_set(dataPrefix + 'new/m_training.npy')
# training_sparse2 = read_sparse_binary_set(dataPrefix + 'm_test2_tzw_nowe_cechy.npy')

test_sparse = read_sparse_binary_set(dataPrefix + 'new/m_test.npy')

cards_types_training = np.load(dataPrefix + 'new2/card_details_ready_matrix_training.npy')
# cards_types_training2 = np.load(dataPrefix + 'card_details_ready_matrix_test2.npy')

# training_sparse = np.concatenate((training_sparse1, training_sparse2), axis = 0)
#  cards_types_training = np.concatenate((cards_types_training1, cards_types_training2), axis = 0)

cards_types_test = np.load(dataPrefix + 'new2/card_details_ready_matrix_test.npy')

#XRawTestDataset = np.concatenate((XRawTestDataset, test_sparse[:,1:40], cards_types_test), axis = 1)
#XRawTrainingDataset = np.concatenate((XRawTrainingDataset, training_sparse[:,1:40], cards_types_training), axis = 1)

XRawTestDataset = np.concatenate((XRawTestDataset, test_sparse[:,1:40]), axis = 1)
XRawTrainingDataset = np.concatenate((XRawTrainingDataset, training_sparse[:,1:40]), axis = 1)

min_max_scaler = preprocessing.MinMaxScaler()

#XTrainingDatasetScaledWithOutliers = dataScaling(XRawTrainingDataset, min_max_scaler)
#XTestDatasetScaled = dataScaling(XRawTestDataset, min_max_scaler)



tuplesTrain = set()
XtrainMap=dict()
YtrainMap=dict()
XtestMap=dict()
i = 0
for entry in XRawTrainingDataset:
    tuplesTrain.add((entry[3], entry[15]))
    toAdd = res = np.hstack([entry[0:3],entry[4:15],entry[16:len(entry)]])
    if (entry[3], entry[15]) in XtrainMap:
        XtrainMap[(entry[3], entry[15])].append(toAdd)
        YtrainMap[(entry[3], entry[15])].append(YRawTrainingDataset[i])
    else:
        XtrainMap[(entry[3], entry[15])] = [toAdd]
        YtrainMap[(entry[3], entry[15])] = [YRawTrainingDataset[i]]
    i+=1
tuplesTest = set()
i = 0
for entry in XRawTestDataset:
    tuplesTest.add((entry[3], entry[15]))  
    toAdd = res = np.hstack([entry[0:3],entry[4:15],entry[16:len(entry)]])
    if (entry[3], entry[15]) in XtestMap:
        XtestMap[(entry[3], entry[15])].append((toAdd,i))
    else:
        XtestMap[(entry[3], entry[15])] = [(toAdd,i)]
    i+=1
m1 = []
m2 = []
m3 = []
for (k1,k2) in XtrainMap:
    if len(m1)<27:
        m1.append((k1,k2))
    elif len(m2)<27:
        m2.append((k1,k2))
    else:
        m3.append((k1,k2))
def partialPrediction(tuple):
    clf = GradientBoostingClassifier(n_estimators=150, max_depth=5, min_samples_split = 50, min_samples_leaf = 5)
    clf = clf.fit(dataScaling(np.array(XtrainMap[tuple]),min_max_scaler), YtrainMap[tuple])
    predictedTestWon = clf.predict_proba(dataScaling(np.array(map(lambda x: x[0], XtestMap[tuple])), min_max_scaler))[:,1]
    return zip(map(lambda x: x[1], XtestMap[tuple]), predictedTestWon)
partialRes3 = []
for tuple in m3:
    partialRes3.append(partialPrediction(tuple))
    
for (k1,k2) in XtrainMap:
    print k1,k2,len(XtrainMap[(k1, k2)]), len(YtrainMap[(k1, k2)])    
    
from sklearn.ensemble import (RandomTreesEmbedding, IsolationForest, RandomForestClassifier, GradientBoostingClassifier)
dataFeaturesPrefix = '/home/data/aaia17/features/'
dataPrefix = '/home/data/aaia17/'

def read_sparse_binary_set(matrix_path):
    return np.load(matrix_path)
def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
XTrain = read_sparse_binary_set(dataFeaturesPrefix + 'train-features-79-old-approach.dat.npy')
XTest  = read_sparse_binary_set(dataFeaturesPrefix + 'test-features-79-old-approach.dat.npy')
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

clf = GradientBoostingClassifier(n_estimators=150, max_depth=5, min_samples_split = 50, min_samples_leaf = 5)
clf = clf.fit(XTrain, labels)
predictedTestWon = clf.predict_proba(XTest)[:,1]
writeToFile(predictedTestWon, dataPrefix+'f1.txt')


trainData = np.empty((0, 113), float)
for i in range(0,10):
    trainData = np.vstack([trainData, read_from_pickle(dataFeaturesOutput+'handsHist-%d.store'%i)])
    
testData = np.empty((0, 113), float)
for i in range(0,5):
    testData = np.vstack([testData, read_from_pickle(dataFeaturesOutput+'handsHistTest-%d.store'%i)])    
    
np.save(dataFeaturesOutput+"trainSet.dat", trainData)
np.save(dataFeaturesOutput+"testSet.dat", testData)
np.save(dataFeaturesOutput+"labels.dat", labels)   

min_max_scaler = preprocessing.MinMaxScaler()
XTrain_t = dataScaling(trainData, min_max_scaler)
XTest_t = dataScaling(testData, min_max_scaler)

dimToRemove = getDimToRemove(XTrain_t, XTest_t)
XTrain = stripDim(XTrain_t, dimToRemove)
XTest = stripDim(XTest_t, dimToRemove)

np.save(dataFeaturesOutput+"trainSetScaled.dat", XTrain)
np.save(dataFeaturesOutput+"testSetScaled.dat", XTest)


chosen = read_from_pickle(dataPrefix+"chosen-64.store")
chosen


trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
trainingFilesDepr = ['testData_tabular_chunk5.csv', 'testData_tabular_chunk6.csv',
                 'testData_tabular_chunk7.csv']
testFiles = ['testData_tabular.csv']
rawTraining = readDataAsFrame(dataPrefix, trainingFiles)
rawTest     = readDataAsFrame(dataPrefix, testFiles)

rawTrainingDepr = readDataAsFrame(dataDepreciatedPrefix, trainingFilesDepr)
labels = read_sparse_binary_set(dataFeaturesPrefix + 'labels.dat.npy')

exclude_features = ['opponent.hero_card_id', 'player.hero_card_id']
rawTrainingDataset = rawTraining.drop(exclude_features, axis = 1)
rawTrainingDeprDataset = rawTrainingDepr.drop(exclude_features, axis = 1)
rawTestDataset = rawTest.drop(exclude_features, axis = 1)

XRawTrain = rawTrainingDataset.values[:,2:43]
XRawTrainDepr = rawTrainingDeprDataset.values[:,2:43]
XRawTest_  = rawTestDataset.values[:,2:43]

XRawTrainAll_ = np.vstack([XRawTrain, XRawTrainDepr])
XRawTrainAll  = np.hstack([XRawTrainAll_, dataTrain])
XRawTest      = np.hstack([XRawTest_, dataTest])

# Make dense features

In [ ]:
columnsToDensify = ['opponent.hero_card_id', 'player.hero_card_id']
chd = computeFractionsForAll(rawTrainingDataset, columnsToDensify)
X_train_denseFeatures = getDenseFeatures(rawTrainingDataset, columnsToDensify, chd)
X_test_denseFeatures  = getDenseFeatures(rawTestDataset, columnsToDensify, chd)

#  Drop categorical or unnecessary features

In [ ]:
YRawTrainingDataset = rawTrainingDataset.values[:,1]
columnsToDrop = ['decision','gamestate_id', 'opponent.hero_card_id', 'player.hero_card_id']
rawDroppedTrainingDataset = dropColumnsFromDataFrame(rawTrainingDataset, columnsToDrop)
rawDroppedTestDataset = dropColumnsFromDataFrame(rawTestDataset, columnsToDrop)

YRawTrainingDataset = rawTrainingDataset.values[:,1]
columnsToDrop = ['decision','gamestate_id']
rawDroppedTrainingDataset = dropColumnsFromDataFrame(rawTrainingDataset, columnsToDrop)
rawDroppedTestDataset = dropColumnsFromDataFrame(rawTestDataset, columnsToDrop)

# Add some features before scaling

In [ ]:
XRawTrainingDataset = rawDroppedTrainingDataset.values.astype(float)
XRawTestDataset     = rawDroppedTestDataset.values.astype(float)

#XRawTrainingDataset = np.concatenate((XRawTrainingDataset, training_sparse[:,0:78]), axis = 1)
#XRawTestDataset = np.concatenate((XRawTestDataset, test_sparse[:,0:78]), axis = 1)
XRawTrainingDataset = np.concatenate((XRawTrainingDataset, training_sparse_New[:,1:40]), axis = 1)
XRawTestDataset = np.concatenate((XRawTestDataset, test_sparse_New[:,1:40]), axis = 1)
#assert np.all(training_sparse[:,78] == rawTrainingDataset.values[:,0]) and  np.all(test_sparse[:,78] == rawTestDataset.values[:,0])
assert np.all(training_sparse_New[:,0] == rawTrainingDataset.values[:,0]) and  np.all(test_sparse_New[:,0] == rawTestDataset.values[:,0])

# Add new features after scaling

In [ ]:
XTrainingDatasetScaledWithOutliers = addNewFeature(XTrainingDatasetScaledWithOutliers, X_train_denseFeatures)
XTestDatasetScaled = addNewFeature(XTestDatasetScaled, X_test_denseFeatures)